# BERT-base-multilingual-cased

In [40]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm  # Barre de progression pour Jupyter
pd.set_option('display.max_colwidth', None)
import s3fs

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.preprocessing import MultiLabelBinarizer
from datasets import Dataset
import re

import torch
from sklearn.metrics import (
    f1_score, precision_score, recall_score, hamming_loss, accuracy_score
)

In [9]:
os.chdir("/home/onyxia/work/projet_NLP")
print(os.getcwd())

/home/onyxia/work/projet_NLP


In [10]:
data_train = pd.read_pickle('data/df_train.pkl')
data_test = pd.read_pickle('data/df_test.pkl')

In [11]:
print(torch.cuda.is_available())  # doit renvoyer True
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))  # nom de ton GPU

True
NVIDIA A2


## Addition of French dataset train to training set


In [ ]:
data_train = data_train.filter(items=['case_text', 'target'])

In [20]:
data_train['target'].head()

1952    [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
3615      [True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False]
1883      [False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False]
4767         [False, False, False, True, False, False, False, False, False, False, False, True, False, False, True, False, False, False, False, True, False, False, True, False, False, False]
576      [False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
Name: target, dtype: object

In [92]:
df_train_fr = pd.read_csv("GEMINI/df_train_fr.csv")
df_train_fr.columns


Index(['Unnamed: 0.1', 'Unnamed: 0', 'Cas_clinique', 'MeSH_Code'], dtype='object')

In [93]:
df_train_fr = df_train_fr.filter(items=['Cas_clinique', 'MeSH_Code'])
df_train_fr = df_train_fr.rename(columns={"Cas_clinique": "case_text", "MeSH_Code": "target"})
df_train_fr.columns

Index(['case_text', 'target'], dtype='object')

In [94]:
import ast

def extract_mesh_codes(lst):
    """
    À partir d'une liste de strings ['C04 – neoplasms', ...],
    renvoie ['04', '08', ...] (sans le 'C').
    """
    codes = []
    for item in lst:
        # on découpe sur l’en-dash « – » ou le tiret-moins « - »
        raw = item.split('–')[0].split('-')[0].strip()
        # si ça commence par 'C', on retire la lettre
        if raw.upper().startswith('C') and len(raw) > 1:
            num = raw[1:]
        else:
            num = raw
        codes.append(num)
    return codes

def process_target_string(s):
    """
    Transforme la chaîne s, qui représente une liste Python,
    en réelle liste, puis en extrait les codes numériques sans 'C'.
    """
    try:
        lst = ast.literal_eval(s)
    except (ValueError, SyntaxError):
        return []
    return extract_mesh_codes(lst)

# Application à votre DataFrame
df_train_fr['target'] = df_train_fr['target'].apply(process_target_string)

# Vérification
print(df_train_fr['target'].head())
# Ex. : [['04', '08', '12'], ['01', '08', '20'], ...]


0    [04, 08, 12]
1    [01, 08, 20]
2        [05, 07]
3    [06, 17, 23]
4        [08, 16]
Name: target, dtype: object


In [88]:
df_train_fr['target'][0]

"['C04 – neoplasms', 'C08 – respiratory tract diseases', 'C12 – urologic and male genital diseases']"

In [ ]:
from Script import multilabel_preprocessing as mp
df_train_fr['target'] = df_train_fr['target'].apply(extract_mesh_codes)


In [95]:
df_train_fr['target'] = df_train_fr['target'].apply(mp.vectorizer)

In [96]:
df_train_fr.head(2)

,case_text,target
0,"Monsieur Dupont, âgé de 72 ans, consulte son médecin traitant pour des troubles urinaires évoluant depuis plusieurs mois. Il se plaint principalement d'une augmentation de la fréquence des mictions, y compris la nuit (nycturie), d'une difficulté à initier la miction et d'un jet urinaire faible. Il note également une sensation de vidange incomplète de la vessie après avoir uriné.\n\nMonsieur Dupont a des antécédents d'hypertension artérielle traitée par inhibiteur de l'enzyme de conversion (IEC) et de tabagisme chronique (40 paquets-années), bien qu'il ait arrêté de fumer il y a 10 ans. Il n'a pas d'antécédents familiaux de cancer de la prostate.\n\nL'examen clinique révèle un abdomen souple et non douloureux. La palpation sus-pubienne ne révèle pas de globe vésical. Le toucher rectal met en évidence une prostate augmentée de volume, de consistance ferme et régulière, sans nodule suspect. Un dosage du PSA (antigène prostatique spécifique) est prescrit et revient à 7,5 ng/mL (valeur normale < 4 ng/mL).\n\nCompte tenu de ces résultats et des symptômes du patient, une consultation urologique est demandée. L'urologue réalise un examen complémentaire comprenant un débitmétrie urinaire qui confirme une diminution significative du débit maximal (Qmax). Une échographie de la vessie avec mesure du résidu post-mictionnel révèle un résidu important (> 150 mL).\n\nFace à la suspicion d'hypertrophie bénigne de la prostate (HBP) associée à un PSA élevé, l'urologue propose la réalisation de biopsies prostatiques échoguidées. Les biopsies révèlent la présence d'un adénocarcinome prostatique de grade Gleason 7 (3+4) dans plusieurs carottes. Un bilan d'extension est réalisé, comprenant une scintigraphie osseuse et une IRM pelvienne, qui ne mettent pas en évidence de métastases à distance.\n\nMonsieur Dupont est informé du diagnostic de cancer de la prostate localisé de risque intermédiaire. Une discussion multidisciplinaire est organisée pour déterminer la meilleure option thérapeutique. Compte tenu de son âge, de ses comorbidités et de ses préférences, une prostatectomie radicale robot-assistée est proposée. L'intervention se déroule sans complications notables. L'examen histopathologique de la pièce opératoire confirme le diagnostic d'adénocarcinome prostatique avec marges chirurgicales saines. Des ganglions lymphatiques pelviens sont prélevés et ne présentent pas de métastases. \n\nEn post-opératoire, Monsieur Dupont présente une incontinence urinaire d'effort transitoire, qui s'améliore progressivement avec des séances de rééducation périnéale. Un suivi régulier du PSA est mis en place pour surveiller l'évolution de la maladie. \n\nTrois ans après l'intervention, Monsieur Dupont consulte pour une dyspnée progressive. Une radiographie thoracique révèle une image suspecte au niveau du poumon droit. Une TDM thoracique confirme la présence d'un nodule pulmonaire. Une biopsie de ce nodule révèle une métastase du cancer de la prostate. Une hormonothérapie est alors initiée pour contrôler la progression de la maladie.","[False, False, False, True, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]"
1,"Mme. Rivière, âgée de 55 ans et présentant des antécédents d'asthme mal contrôlé et d'allergie aux acariens, s'est présentée à la clinique avec une toux persistante, une dyspnée croissante et une fièvre légère durant depuis environ une semaine. Elle a également rapporté une sensation de serrement thoracique et une production accrue de mucus jaunâtre. Ses symptômes se sont progressivement aggravés malgré l'utilisation de son inhalateur de secours (salbutamol). \n\nÀ l'examen clinique, on notait une respiration sifflante diffuse et des râles crépitants aux deux bases pulmonaires. Sa saturation en oxygène était à 90% sous air ambiant. Une radiographie thoracique a révélé des infiltrats bilatéraux, plus prononcés dans les lobes inférie

In [97]:
df_train_fr.to_pickle("data/df_train_fr.pkl")

## Training BERT-base-multilingual-cased

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Modèle bilingue FR/EN (multilingual BERT)
model_name = "bert-base-multilingual-cased"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Nombre de classes en sortie
n_labels = 26  # à ajuster selon ton jeu de données

# Modèle pour classification multi-label
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=n_labels,
    problem_type="multi_label_classification"
)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# 1. Calculer le nombre de tokens pour chaque case_text
stat=pd.DataFrame()
stat['n_tokens'] = data_train['case_text'].apply(lambda x: len(tokenizer.tokenize(str(x))))

# 2. Afficher les stats descriptives
print(stat['n_tokens'].describe())

Token indices sequence length is longer than the specified maximum sequence length for this model (794 > 512). Running this sequence through the model will result in indexing errors


count     9646.000000
mean       908.879121
std        641.935476
min         34.000000
25%        501.000000
50%        770.000000
75%       1128.000000
max      14186.000000
Name: n_tokens, dtype: float64


In [ ]:
from datasets import Dataset
df = data_train.rename(columns={"target": "labels"})
df["labels"] = df["labels"].apply(lambda x: np.array(x, dtype=np.float32))
dataset = Dataset.from_pandas(df)

# Tokenizer
def tokenize(batch):
    return tokenizer(batch["case_text"], padding="max_length", truncation=True, max_length=512)

dataset = dataset.map(tokenize, batched=True)

Map: 100%|██████████| 10718/10718 [00:04<00:00, 2313.24 examples/s]


In [15]:
from datasets import DatasetDict
dataset_train_val = dataset.train_test_split(test_size=0.1/0.9) # 0.1 /0.9 pour avoir meme taille de validation set et de test set

print("taille dataset entrainement :", dataset_train_val["train"].shape[0])
print("taille dataset validation :", dataset_train_val["test"].shape[0])

NameError: name 'dataset' is not defined

In [ ]:
training_args = TrainingArguments(
    output_dir="../results_BERT_ml",
    eval_strategy="epoch",              # évaluation à chaque époque
    save_strategy="epoch",                    # sauvegarde à chaque époque (utile pour reprise)
    save_total_limit=2,                       # limite le nombre de checkpoints
    learning_rate=3e-5,                       # légèrement augmenté pour convergence plus rapide
    per_device_train_batch_size=4,            # réduit à 4 pour éviter OOM (GPU 16 Go)
    per_device_eval_batch_size=4,
    num_train_epochs=5,                       # plus d'époques si dataset pas trop gros
    weight_decay=0.01,
    fp16=True,                                # active le mixed precision training (optimisé pour Ampere)
    gradient_accumulation_steps=4,            # simule un batch size plus grand (4x4 = 16)
    warmup_ratio=0.1,                         # warmup sur 10% des steps pour stabiliser l'entraînement
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,              # récupère le meilleur modèle (selon eval loss)
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    dataloader_num_workers=2,                 # légère parallélisation de l’I/O
    report_to="none",                         # désactive Weights & Biases si non utilisé
)

def exact_match_accuracy(y_true, y_pred):
    return np.all(y_true == y_pred, axis=1).mean()

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    thresholds = np.linspace(0.1, 0.99, 20)

    best_threshold = 0.5
    best_precision = 0

    for t in thresholds:
        preds = (logits >= t).astype(int)
        micro_precision = precision_score(labels, preds, average='micro', zero_division=0)
        if micro_precision > best_precision:
            best_precision = micro_precision
            best_threshold = t

    # Final prediction with the best threshold
    preds = (logits >= best_threshold).astype(int)

    return {
        'threshold': best_threshold,
        'micro_precision': precision_score(labels, preds, average='micro', zero_division=0),
        'micro_recall': recall_score(labels, preds, average='micro', zero_division=0),
        'micro_f1': f1_score(labels, preds, average='micro', zero_division=0),
        'hamming_loss': hamming_loss(labels, preds),
        'exact_match_accuracy': exact_match_accuracy(labels, preds)
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train_val["train"],
    eval_dataset=dataset_train_val["test"],
    compute_metrics=compute_metrics
)

trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# 1. Préparez votre DataFrame de test :
#    - Renommez la colonne target en labels
#    - Transformez chaque liste de labels en array float32
df_test = data_test.rename(columns={"target": "labels"})
df_test["labels"] = df_test["labels"].apply(lambda x: np.array(x, dtype=np.float32))

# 2. Créez un Dataset Hugging Face
test_dataset = Dataset.from_pandas(df_test)

# 3. Définissez la même fonction de tokenisation que pour l’entraînement
def tokenize(batch):
    return tokenizer(
        batch["case_text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

# 4. Appliquez la tokenisation
test_dataset = test_dataset.map(tokenize, batched=True)

# 5. Facultatif : fixez le format PyTorch pour éviter d’avoir à convertir à la main
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# 6a. Évaluation simple : renvoie loss + métriques de compute_metrics
test_metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Résultats sur test :", test_metrics)

In [ ]:
trainer.train(resume_from_checkpoint="./results/checkpoint-3350")

Epoch,Training Loss,Validation Loss


TrainOutput(global_step=3350, training_loss=0.0, metrics={'train_runtime': 0.4082, 'train_samples_per_second': 131291.268, 'train_steps_per_second': 8207.235, 'total_flos': 1.4136845312249856e+16, 'train_loss': 0.0, 'epoch': 5.0})

In [ ]:
metrics = trainer.evaluate()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
df = pd.DataFrame([metrics])
display(df)

,eval_loss,eval_threshold,eval_micro_precision,eval_micro_recall,eval_micro_f1,eval_hamming_loss,eval_exact_match_accuracy,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,0.052049,0.99,0.994627,0.777871,0.872996,0.022227,0.564658,115.6827,92.65,23.167,5.0


In [ ]:
trainer.save_model(".././mon_modele_final2")
tokenizer.save_pretrained(".././mon_modele_final2")

('./mon_modele_final/tokenizer_config.json',
 './mon_modele_final/special_tokens_map.json',
 './mon_modele_final/vocab.txt',
 './mon_modele_final/added_tokens.json',
 './mon_modele_final/tokenizer.json')

## TEST du modèle

In [ ]:
text = """
A 68-year-old man presents with a persistent cough, present for three months, accompanied by increasing shortness of breath when he exerts himself. He also complains of recent lower back pain.
He has a significant smoking history, having smoked the equivalent of 40 packs of cigarettes per year. Notably, he reports coughing up sputum tinged with blood on occasion. During the physical examination, 
the physician observes diminished breath sounds specifically in the lower portion of his right lung. An initial chest X-ray reveals a concerning mass located in the right lower lobe of the lung.
To further investigate, a CT scan of the chest is performed. This imaging confirms the presence of a 4-centimeter mass within the right lower lobe. Additionally, 
the scan reveals enlarged lymph nodes in the region of the lung's hilum (hilar lymphadenopathy). Upon further questioning, the patient admits to experiencing nocturia, characterized by the need to urinate frequently during the night, 
approximately two to three times per night, over the past six months. He initially attributed this to simply drinking more fluids before bed. He also mentions mild, intermittent lower back pain that sometimes radiates down his right leg. 
He had previously dismissed this pain as a normal consequence of aging and stiffness. His medical history includes high blood pressure (hypertension), which is currently being managed with medication. An electrocardiogram (ECG) is performed as part of the evaluation. 
The ECG reveals a left bundle branch block, which is a new finding compared to previous ECG recordings. An echocardiogram shows mild left ventricular hypertrophy. To determine the specific nature of the lung mass and assess the involvement of the lymph nodes, 
the patient is scheduled for a bronchoscopy with a biopsy. In addition, due to his reported nocturia and lower back pain, 
a prostate-specific antigen (PSA) test will be performed to evaluate prostate health. A more comprehensive cardiac assessment is planned to further investigate the newly identified left bundle branch block.
"""

In [ ]:
text

"\nA 68-year-old man presents with a persistent cough, present for three months, accompanied by increasing shortness of breath when he exerts himself. He also complains of recent lower back pain.\nHe has a significant smoking history, having smoked the equivalent of 40 packs of cigarettes per year. Notably, he reports coughing up sputum tinged with blood on occasion. During the physical examination, \nthe physician observes diminished breath sounds specifically in the lower portion of his right lung. An initial chest X-ray reveals a concerning mass located in the right lower lobe of the lung.\nTo further investigate, a CT scan of the chest is performed. This imaging confirms the presence of a 4-centimeter mass within the right lower lobe. Additionally, \nthe scan reveals enlarged lymph nodes in the region of the lung's hilum (hilar lymphadenopathy). Upon further questioning, the patient admits to experiencing nocturia, characterized by the need to urinate frequently during the night, \

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("./mon_modele_final2")
tokenizer = AutoTokenizer.from_pretrained("./mon_modele_final2")

In [ ]:
# Tokenization
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

# Mettre le modèle en mode évaluation
model.eval()

# Tokenisation
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

# Prédiction (désactive le calcul de gradients)
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Appliquer une sigmoïde pour obtenir les probabilités
probs = torch.sigmoid(logits)

# Seuil pour dire si chaque label est actif ou pas (ici 0.5)
predicted_labels = (probs > 0.5).squeeze().bool().tolist()

# Affichage
print(predicted_labels)

[False, False, False, True, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False]


In [ ]:
type(predicted_labels)

list

In [ ]:
import multilabel_preprocessing as mp
mp.mesh_labels_from_vector(np.array(predicted_labels))

['C04 – neoplasms',
 'C08 – respiratory tract diseases',
 'C14 – cardiovascular diseases']

## Autre

| Étape                  | Outils                         | Ce que tu fais                            |
|------------------------|--------------------------------|--------------------------------------------|
| Choix du modèle        | HuggingFace `transformers`     | Utilise un BERT médical pré-entraîné       |
| Préparation des données| `datasets`, `tokenizer`        | Tokenisation + conversion des labels       |
| Modélisation           | `AutoModelForSequenceClassification` | Déclare une classification multi-label |
| Entraînement           | `Trainer`                      | Fine-tuning du modèle sur tes données      |
| Évaluation             | `f1_score`, `hamming_loss`     | Calcul des performances globales           |

In [ ]:
raise RuntimeError("⛔ Cette cellule ne doit pas être exécutée.")

# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

BUCKET_OUT = "s3://quentin1999/Data_Projet_NLP"
FILE_KEY_OUT_S3 = "df_target_V3.pkl"
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'wb') as file_out:
    df.to_pickle(file_out)

In [ ]:
raise RuntimeError("⛔ Cette cellule ne doit pas être exécutée.")

from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("./mon_modele_final")
tokenizer = AutoTokenizer.from_pretrained("./mon_modele_final")

## Enregistrer le modèle BERT trained

In [ ]:
import shutil
import zipfile

# === 1. Zippage du dossier ===
output_dir = ".././mon_modele_final"
zip_path = ".././mon_modele_final.zip"
shutil.make_archive(base_name="mon_modele_final", format='zip', root_dir=output_dir)

# === 2. Envoi vers S3 Vault ===
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

BUCKET_OUT = "s3://quentin1999/Data_Projet_NLP"
MODEL_ZIP_KEY = "mon_modele_final.zip"
MODEL_ZIP_PATH_S3 = BUCKET_OUT + "/" + MODEL_ZIP_KEY

with fs.open(MODEL_ZIP_PATH_S3, 'wb') as f_out:
    with open(zip_path, 'rb') as f_in:
        shutil.copyfileobj(f_in, f_out)

print("✅ Modèle sauvegardé dans le Vault S3 :", MODEL_ZIP_PATH_S3)

✅ Modèle sauvegardé dans le Vault S3 : s3://quentin1999/Data_Projet_NLP/mon_modele_final.zip


In [ ]:
# Téléchargement depuis S3
with fs.open(MODEL_ZIP_PATH_S3, 'rb') as f_in:
    with open("mon_modele_final.zip", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Dézippage
shutil.unpack_archive("mon_modele_final.zip", extract_dir="./mon_modele_final")

# Chargement
model = AutoModelForSequenceClassification.from_pretrained("./mon_modele_final")
tokenizer = AutoTokenizer.from_pretrained("./mon_modele_final")

In [ ]:
from sklearn.metrics import f1_score, hamming_loss
import numpy as np
import torch
from tqdm import tqdm

# 1. Mettre le modèle en mode évaluation
model.eval()

# Exemple si tu as X_test sous forme de textes
for batch in tqdm(X_test):  # ou DataLoader, selon ta structure
    inputs = tokenizer(batch, return_tensors="pt", truncation=True, padding=True).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits.cpu().numpy()
    y_pred_logits.append(logits)

# 3. Empiler les logits et binariser
y_pred_logits = np.vstack(y_pred_logits)         # (n_samples, n_classes)
y_pred = (y_pred_logits >= 0.5).astype(int)      # Seuil de 0.5 pour binariser

# 4. Évaluation (en supposant que y_test_array est déjà binairisé)
f1_micro = f1_score(y_test_array, y_pred, average='micro')
f1_macro = f1_score(y_test_array, y_pred, average='macro')
hamming = hamming_loss(y_test_array, y_pred)

print(f"✅ F1 Micro : {f1_micro:.4f}")
print(f"✅ F1 Macro : {f1_macro:.4f}")
print(f"🔁 Hamming Loss : {hamming:.4f}")

NameError: name 'X_test' is not defined